In [1]:
import xlwings as xw
import pandas as pd

from Options_Math_Helpers import *
from Options_Math_Algebra import *
from Options_Math_Black_Scholes import *

oma  = OptionsMathAlgebra()
ombs = OptionsMathBlackScholes()

In [2]:
# Open workbook
wb = xw.Book('deribit.xlsx')

# Select sheet by name
sheet = wb.sheets['combo_with_time']

# Read entire used range into a DataFrame
raw_df = sheet.used_range.options(pd.DataFrame, header=True, index=False).value

In [3]:
cols = ['kind', 
        'instrument_name', 
        'option_type', 
        'strike', 
        'expiration_time_nyc', 
        'years_to_expiry', 
        'underlying_price',
        'mark_price', 
        'best_bid_price', 
        'best_ask_price',
        'mark_iv', 
        'bid_iv', 
        'ask_iv'
       ]

df = raw_df.loc[:, [c for c in cols if c in raw_df.columns]]
df = df.loc[df['kind'] == 'option']
df = df.drop(columns=['kind'])

print('done')

done


In [4]:
opt_types = df['option_type']
fwds      = df['underlying_price']
strikes   = df['strike']
times     = df['years_to_expiry'] #* 365/365.25

df['mark_dollar']     = df['underlying_price'] * df['mark_price']
df['best_bid_dollar'] = df['underlying_price'] * df['best_bid_price']  
df['best_ask_dollar'] = df['underlying_price'] * df['best_ask_price'] 

In [5]:
df['mark_iv_mine'] = ombs.bs_vol_solver_newton(opt_value=df['mark_dollar'], opt_type=opt_types, fwd=fwds, strike=strikes, time=times)
#df['mark_iv_mine'] = ombs.bs_vol_solver_brentq(opt_value=df['best_ask_dollar'], opt_type=opt_types, fwd=fwds, strike=strikes, time=times)
df['mark_iv_mine'] = df['mark_iv_mine'] * 100

df['bid_iv_mine'] = ombs.bs_vol_solver_newton(opt_value=df['best_bid_dollar'], opt_type=opt_types, fwd=fwds, strike=strikes, time=times)
#df['bid_iv_mine'] = ombs.bs_vol_solver_brentq(opt_value=df['best_bid_dollar'], opt_type=opt_types, fwd=fwds, strike=strikes, time=times)
df['bid_iv_mine'] = df['bid_iv_mine'] * 100

df['ask_iv_mine'] = ombs.bs_vol_solver_newton(opt_value=df['best_ask_dollar'], opt_type=opt_types, fwd=fwds, strike=strikes, time=times)
#df['ask_iv_mine'] = ombs.bs_vol_solver_brentq(opt_value=df['best_ask_dollar'], opt_type=opt_types, fwd=fwds, strike=strikes, time=times)
df['ask_iv_mine'] = df['ask_iv_mine'] * 100

C:\Users\micha\OneDrive\Deribit\Options_Math_Black_Scholes.py:387: RuntimeWarning: divide by zero encountered in divide
  safe_step = np.where(np.abs(vega) > 1e-8, 0.01 * diff / vega, 0.0)
C:\Users\micha\OneDrive\Deribit\Options_Math_Black_Scholes.py:387: RuntimeWarning: divide by zero encountered in divide
  safe_step = np.where(np.abs(vega) > 1e-8, 0.01 * diff / vega, 0.0)
C:\Users\micha\OneDrive\Deribit\Options_Math_Black_Scholes.py:387: RuntimeWarning: divide by zero encountered in divide
  safe_step = np.where(np.abs(vega) > 1e-8, 0.01 * diff / vega, 0.0)


In [6]:
df['mark_iv_diff'] = df['mark_iv_mine'] - df['mark_iv'].astype(float)
df['bid_iv_diff']  = df['bid_iv_mine']  - df['bid_iv'].astype(float)
df['ask_iv_diff']  = df['ask_iv_mine']  - df['ask_iv'].astype(float)

In [7]:
sheet = wb.sheets('vols')
sheet.range('a1').options(index = False, header=1).value = df